In [36]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Load data

In [3]:
data = pd.read_csv("token_datasets.csv")
data.head()

,Posts,label,tokenized_text,filtered_text,lemmatized_text
0,i m still live at home with mom and her bf whi...,Positive,"['i', 'm', 'still', 'live', 'at', 'home', 'wit...","['still', 'live', 'home', 'mom', 'bf', 'pursue...","['still', 'live', 'home', 'mom', 'bf', 'pursue..."
1,i am honestly on the verge of self harm i cant...,Neutral,"['i', 'am', 'honestly', 'on', 'the', 'verge', ...","['honestly', 'verge', 'self', 'harm', 'cant', ...","['honestly', 'verge', 'self', 'harm', 'cant', ..."
2,sorry that this is long its complicated every ...,Positive,"['sorry', 'that', 'this', 'is', 'long', 'its',...","['sorry', 'long', 'complicated', 'every', 'tim...","['sorry', 'long', 'complicated', 'every', 'tim..."
3,my life has been a train wreck im only now and...,Positive,"['my', 'life', 'has', 'been', 'a', 'train', 'w...","['life', 'train', 'wreck', 'im', 'ive', 'alrea...","['life', 'train', 'wreck', 'im', 'ive', 'alrea..."
4,he can be such a controlling asshole who i can...,Neutral,"['he', 'can', 'be', 'such', 'a', 'controlling'...","['controlling', 'asshole', 'cant', 'say', 'any...","['controlling', 'asshole', 'cant', 'say', 'any..."


In [5]:
data = data.drop(['Posts','tokenized_text','filtered_text'], axis=1)
data.head()

,label,lemmatized_text
0,Positive,"['still', 'live', 'home', 'mom', 'bf', 'pursue..."
1,Neutral,"['honestly', 'verge', 'self', 'harm', 'cant', ..."
2,Positive,"['sorry', 'long', 'complicated', 'every', 'tim..."
3,Positive,"['life', 'train', 'wreck', 'im', 'ive', 'alrea..."
4,Neutral,"['controlling', 'asshole', 'cant', 'say', 'any..."


In [27]:
vectorizer = CountVectorizer()
vec_transformed = vectorizer.fit_transform(data.lemmatized_text)
DocTermMatrix = pd.DataFrame(vec_transformed.toarray(), columns=vectorizer.get_feature_names_out())
DocTermMatrix.head()

,aaaaa,aaahhhhhh,aaron,ab,abandon,abandoned,abandoning,abandonment,abandonned,abc,...,zigzag,zobble,zoim,zombie,zomble,zone,zoned,zoo,zookeepers,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
labelEncoder = LabelEncoder()
labelling_transformed = labelEncoder.fit_transform(labelling)

In [38]:
labelling = data.iloc[:,0]
labelling.head(10)

0                        Positive
1                         Neutral
2                        Positive
3                        Positive
4                         Neutral
5    Depression/Suicidal Thoughts
6                        Positive
7                         Neutral
8                         Neutral
9                        Positive
Name: label, dtype: object

In [41]:
labelling_transformed[:10]

array([3, 2, 3, 3, 2, 0, 3, 2, 2, 3])

In [43]:
list(labelEncoder.inverse_transform([0,1,2,3]))

['Depression/Suicidal Thoughts', 'Mixed Features', 'Neutral', 'Positive']